In [26]:
import pandas as pd


json_data = [
  {
    "name": "John Doe",
    "employee_id": "2021001",
    "id": "42663b29-1d06-417a-b13e-9269229e32a8",
    "role": "admin"
    
  },
  {
    "name": "Jane Smith",
    "id": "27f4d012-2785-4712-8295-20f16e2ab34c",
    "employee_id": "2023002",
    "role": "user_compliance"
    
  },
  {
    "name": "Alice Johnson",
    "id": "e3a3e04b-10e1-486c-b030-d52e4e92b4fb",
    "employee_id": "2024009",
    "role": "user_branch"
    
  }
]

In [27]:
pd.DataFrame(json_data)

,name,employee_id,id,role
0,John Doe,2021001,42663b29-1d06-417a-b13e-9269229e32a8,admin
1,Jane Smith,2023002,27f4d012-2785-4712-8295-20f16e2ab34c,user_compliance
2,Alice Johnson,2024009,e3a3e04b-10e1-486c-b030-d52e4e92b4fb,user_branch


In [28]:
user_id = '2023002'

In [29]:
role = [{"name":row['name'],"role":row['role']} for row in json_data if row['employee_id']==user_id]
role

[{'name': 'Jane Smith', 'role': 'user_compliance'}]

# Deployment

In [30]:
payload = {
    "input_data": [{
        "fields": ["user_id"],
        "values": [["2023002"]]
}]}
payload

{'input_data': [{'fields': ['user_id'], 'values': [['2023002']]}]}

## Deployment Function for Extension

In [31]:
def get_user_info():
    import json
    import requests
    import pandas as pd
    
   
    def get_user_info(payload):      
            ######### User Info ############
        json_data = [
              {
                "name": "John Doe",
                "employee_id": "2021001",
                "id": "42663b29-1d06-417a-b13e-9269229e32a8",
                "role": "admin"

              },
              {
                "name": "Jane Smith",
                "id": "27f4d012-2785-4712-8295-20f16e2ab34c",
                "employee_id": "2023002",
                "role": "user_compliance"

              },
              {
                "name": "Alice Johnson",
                "id": "e3a3e04b-10e1-486c-b030-d52e4e92b4fb",
                "employee_id": "2024009",
                "role": "user_branch"

              }
            ]
       
        
        for data in payload["input_data"]:
            user_id = data['values'][0][0]
            
            user_info = [{"name":row['name'],"role":row['role']} for row in json_data if row['employee_id']==user_id]
        
        return {'predictions': [{'fields': ['result'], 'values': user_info}]}
            
    
    return get_user_info

In [32]:
get_user_info()(payload)

/opt/conda/envs/Python-RT23.1/lib/python3.10/site-packages/urllib3/connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host 'iam.cloud.ibm.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


{'predictions': [{'fields': ['result'],
   'values': [{'name': 'Jane Smith', 'role': 'user_compliance'}]}]}

## Provide Watson Machine Learning (WML) Information

In [33]:
api_key = 'IAM-APIKEY'
location = 'us-south'
space_id= "SPACE_ID_UID"

In [34]:
wml_credentials = {
    "apikey": api_key,
    "url": 'https://' + location + '.ml.cloud.ibm.com'
}

In [35]:
from ibm_watson_machine_learning import APIClient
import requests
import json

client = APIClient(wml_credentials)

In [36]:
client.set.default_space(space_id)
sofware_spec_uid = client.software_specifications.get_id_by_name("runtime-23.1-py3.10")
meta_data = {
    client.repository.FunctionMetaNames.NAME: 'Authenticate user',
    client.repository.FunctionMetaNames.SOFTWARE_SPEC_UID: sofware_spec_uid
}
function_details = client.repository.store_function(meta_props=meta_data, function=get_user_info)

In [37]:
function_uid = client.repository.get_function_uid(function_details)
# Deploy the stored function

metadata = {
    client.deployments.ConfigurationMetaNames.NAME: "Authenticate user",
    client.deployments.ConfigurationMetaNames.ONLINE: {}
}
function_deployment_details = client.deployments.create(function_uid, meta_props=metadata)



#######################################################################################

Synchronous deployment creation for uid: 'c9863858-251d-4045-ac87-a07715d689aa' started

#######################################################################################


initializing
Note: online_url and serving_urls are deprecated and will be removed in a future release. Use inference instead.
..
ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='85dc3dc5-7e50-4e14-8c3f-4429e1f132de'
------------------------------------------------------------------------------------------------




## Test if the deployment space works

In [38]:
# Get the endpoint URL of the function deployment just created
function_deployment_id = client.deployments.get_uid(function_deployment_details)
function_deployment_endpoint_url = client.deployments.get_scoring_href(function_deployment_details)
print("Function deployment id: {}".format(function_deployment_id))
print("Endpoint URL: {}".format(function_deployment_endpoint_url))

Function deployment id: 85dc3dc5-7e50-4e14-8c3f-4429e1f132de
Endpoint URL: https://us-south.ml.cloud.ibm.com/ml/v4/deployments/85dc3dc5-7e50-4e14-8c3f-4429e1f132de/predictions


In [39]:
result = client.deployments.score(function_deployment_id, payload)
if "error" in result:
    print(result["error"])
else:
    print(result)

{'predictions': [{'fields': ['result'], 'values': [{'name': 'Jane Smith', 'role': 'user_compliance'}]}]}
